In [4]:
import requests
import pandas as pd
from pandas import json_normalize
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import numpy as np
import re
import json
from pathlib import Path
import xmltodict

In [5]:
# Si es notebook, mejor usar:
BASE_DIR = Path.cwd()

# Ir un nivel arriba (de notebooks → raíz del repo)
ROOT = BASE_DIR.parent

# Carpeta data dentro del repo
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

In [3]:
url = "https://www.camara.cl/diputados/diputados.aspx#mostrarDiputados"
response = requests.get(url)

# Verificamos que la respuesta sea exitosa (200 = OK)
if response.status_code == 200:
    print("Página descargada con éxito")
else:
    print("Error al acceder:", response.status_code)

Página descargada con éxito


In [4]:
soup = BeautifulSoup(response.text, "html.parser")

In [5]:
contenedor = soup.find("div", id="ContentPlaceHolder1_ContentPlaceHolder1_pnlDiputadosLista")

# Dentro del div, buscar todos los artículos con clase "grid-2"
articulos = contenedor.find_all("article", class_="grid-2")

In [19]:
base_url = "https://www.camara.cl/diputados/"

diputados_data = {}

# Recorrer cada artículo
for art in articulos:
    link = art.find("a")
    if link and link.has_attr("href"):
        href = link["href"]  # ej: detalle/mociones.aspx?prmID=1096
        match = re.search(r"prmID=(\d+)", href)
        if not match:
            continue

        dip_id = match.group(1)
        print("Procesando ID:", dip_id)

        # Construir URL de biografía
        bio_url = f"https://www.camara.cl/diputados/detalle/biografia.aspx?prmId={dip_id}#ficha-diputados"
        detalle_res = requests.get(bio_url)
        detalle_soup = BeautifulSoup(detalle_res.text, "html.parser")

        # Diccionario de secciones de este diputado
        bio_dict = {}

        # Buscar div de biografía
        bio_div = detalle_soup.find("div", class_="biografia")

        if bio_div:
            # Buscar todos los párrafos
            for p in bio_div.find_all("p"):
                span = p.find("span")
                if span:
                    titulo = span.get_text(strip=True)  # ej: "Estudios"
                    span.extract()  # quitar el <span> para quedarnos con solo el contenido
                    contenido = p.get_text(" ", strip=True)  # limpiar el texto
                    bio_dict[titulo] = contenido
        
        dis_div = detalle_soup.find("div", class_="grid-3 aleft m-left14")
        
        if dis_div:
            for p in dis_div.find_all("p"):  # aquí el fix
                lineas = list(p.stripped_strings)

                for l in lineas:
                    if "Distrito" in l:
                        bio_dict["Distrito"] = l.split(":")[1].strip()
                    elif "Región" in l:
                        bio_dict["Región"] = l.split(":")[1].strip()
                    elif "Partido" in l:
                        bio_dict["Partido"] = l.split(":")[1].strip()
        # Guardar en el diccionario general
        diputados_data[dip_id] = bio_dict

# Mostrar resultado en formato JSON (bonito)
print(json.dumps(diputados_data, indent=2, ensure_ascii=False))

Procesando ID: 1096
Procesando ID: 1097
Procesando ID: 1098
Procesando ID: 1009
Procesando ID: 803
Procesando ID: 1099
Procesando ID: 1100
Procesando ID: 1101
Procesando ID: 1102
Procesando ID: 1103
Procesando ID: 1104
Procesando ID: 1012
Procesando ID: 1105
Procesando ID: 1185
Procesando ID: 1106
Procesando ID: 1107
Procesando ID: 1108
Procesando ID: 1109
Procesando ID: 971
Procesando ID: 1013
Procesando ID: 1110
Procesando ID: 815
Procesando ID: 1111
Procesando ID: 1112
Procesando ID: 1113
Procesando ID: 1015
Procesando ID: 1114
Procesando ID: 1016
Procesando ID: 1116
Procesando ID: 973
Procesando ID: 1017
Procesando ID: 1117
Procesando ID: 1019
Procesando ID: 1184
Procesando ID: 1021
Procesando ID: 975
Procesando ID: 1022
Procesando ID: 1118
Procesando ID: 976
Procesando ID: 1119
Procesando ID: 1120
Procesando ID: 1121
Procesando ID: 1122
Procesando ID: 1123
Procesando ID: 1025
Procesando ID: 1125
Procesando ID: 1126
Procesando ID: 1027
Procesando ID: 1028
Procesando ID: 1030
Proces

In [20]:
df_bio = pd.DataFrame.from_dict(diputados_data, orient="index")

# El índice actual son los IDs ("1096", "1097", …), lo paso a columna
df_bio = df_bio.reset_index().rename(columns={"index": "id_diputado"})
df_bio

,id_diputado,Antecedentes Familiares,Actividad Política,Actividad Parlamentaria,Distrito,Región,Partido,Profesion/Actividad,Estudios,Actividad Laboral,Otras actividades a destacar,Actividades Profesionales,Actividades Academicas,Idiomas,Publicaciones,Actividades Gremiales,Actividades preferidas,Resumen Ejecutivo
0,1096,▪ Casada. Tiene una hija y un hijo.,▪ Inicia su trayectoria política en las Juvent...,▪ En noviembre de 2021 es electa diputada para...,Nº 20,Región del Bío Bío,Partido Comunista,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1097,▪ Casado.,▪ Inicia su trayectoria política al incorporar...,▪ En noviembre de 2021 es electo diputado par...,Nº 20,Región del Bío Bío,Partido Demócrata Cristiano,▪ Administrador Público.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1098,▪ Es madre de 4 hijas.,"▪ 2019. Inició su trayectoria política en el ""...",▪ En noviembre de 2021 es electa diputada para...,Nº 3,Región de Antofagasta,Partido Social Cristiano,▪ Estudios Técnicos Agropecuarios.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1009,"▪ Casado. Tiene dos hijos, Jorge y Elena.","▪ Concejal, Municipalidad de Santiago. Período...",▪ En noviembre de 2017 es electo diputado para...,Nº 10,Región Metropolitana de Santiago,Unión Demócrata Independiente,▪ Abogado,▪ Colegio Internacional Nido de Águilas. ▪ Der...,"▪ 2003 - 2017. Emprendedor, empresario indepen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,803,▪ Casado con María Erita Vera. Tiene 3 hijos.,NaN,▪ Diputado en el Periodo Legislativo 2006-2010...,Nº 27,Región de Aysén del General Carlos Ibáñez del ...,Independientes,▪ Obrero de la construcción.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,948,NaN,NaN,▪ Diputado electo Periodos Legislativos 2010-2...,Nº 6,Región de Valparaíso,Independientes,▪ Abogado.,NaN,NaN,"▪ Columnista de los diarios El Observador, El ...",▪ 2006-2009. Abogado independiente en la prov...,NaN,NaN,NaN,NaN,NaN,NaN
151,1068,NaN,NaN,▪ En noviembre de 2017 resultó electa diputada...,Nº 7,Región de Valparaíso,Frente Amplio,▪ Administradora Pública.,"▪ Administración Pública, Universidad de Chile...",NaN,NaN,NaN,NaN,NaN,NaN,▪ 2016. Presidenta de la Federación de Estudi...,NaN,NaN
152,1169,NaN,NaN,▪ En noviembre de 2021 es electa diputada para...,Nº 21,Región del Bío Bío,Frente Amplio,▪ Abogada.,▪ 1975. Enseñanza Secundaria. Liceo de Hombre...,▪ 1983 - 1988. Secretaria y Gestora Jurídica. ...,"▪ 1995-1996. Secretaria de Actas Directorio ""U...",▪ 1992 - 2000. Defensora de Oficio de la Corte...,▪ 1990. Docente asistente en materia de Derech...,NaN,NaN,NaN,NaN,NaN
153,1173,NaN,NaN,▪ En noviembre de 2021 fue electa diputada par...,Nº 12,Región Metropolitana de Santiago,Partido Comunista,NaN,"▪ Estudiante de Administración Pública,",NaN,"▪ Dirigenta estudiantil, feminista.",NaN,NaN,NaN,NaN,▪ 2015. Integra la Comisión Nacional de Estud...,NaN,NaN


In [21]:
df_dip = pd.read_csv(DATA_DIR / "2022-2026" / "diputados.csv")
df_dip

,periodo_id,Periodo,diputado_id,nombre_completo,fecha_nacimiento,sexo,edad,distrito_numero,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino
0,10,2022-2026,1165,Agustín Romero Leiva,1975-01-03,Masculino,47,NaN,PREP,Partido Republicano,0,2022-03-11,2026-03-10 23:59:59
1,10,2022-2026,1179,Alberto Undurraga Vicuña,1969-06-17,Masculino,52,NaN,DC,Partido Demócrata Cristiano,0,2022-03-11,2026-03-10 23:59:59
2,10,2022-2026,1160,Alejandra Placencia Cabello,1978-02-14,Femenino,44,NaN,PC,Partido Comunista,0,2022-03-11,2026-03-10 23:59:59
3,10,2022-2026,1013,Alejandro Bernales Maldonado,1979-02-02,Masculino,43,NaN,LIBERAL,Partido Liberal de Chile,1,2022-03-11,2026-03-10 23:59:59
4,10,2022-2026,1076,Alexis Sepúlveda Soto,1969-12-29,Masculino,52,NaN,PR,Partido Radical de Chile,1,2022-03-11,2026-03-10 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,10,2022-2026,1098,Yovana Ahumada Palma,1973-02-24,Femenino,49,NaN,PSC,Partido Social Cristiano,0,2025-03-17,2026-03-10 23:59:59
224,10,2022-2026,1098,Yovana Ahumada Palma,1973-02-24,Femenino,49,NaN,PDG,Partido de la Gente,0,2022-03-11,2023-04-18 23:59:59
225,10,2022-2026,1017,Álvaro Carter Fernández,1980-10-09,Masculino,41,NaN,IND,Independientes,2,2025-03-19,2026-03-10 23:59:59
226,10,2022-2026,1017,Álvaro Carter Fernández,1980-10-09,Masculino,41,NaN,UDI,Unión Demócrata Independiente,2,2018-03-11,2025-03-18 23:59:59


In [69]:
df_bio["id_diputado"] = df_bio["id_diputado"].astype(str)
df_dip["diputado_id"] = df_dip["diputado_id"].astype(str)

df_bio_enriquecido = df_bio.merge(
    df_dip,
    left_on="id_diputado",
    right_on="diputado_id",
    how="left"  # <– mantiene solo los diputados de df_bio
)

In [70]:
len(df_bio_enriquecido["diputado_id"])

226

In [71]:
df_bio_enriquecido.head(3)

,id_diputado,Antecedentes Familiares,Actividad Política,Actividad Parlamentaria,Distrito,Región,Partido,Profesion/Actividad,Estudios,Actividad Laboral,...,fecha_nacimiento,sexo,edad,distrito_numero,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino,llave_y
0,1096,▪ Casada. Tiene una hija y un hijo.,▪ Inicia su trayectoria política en las Juvent...,▪ En noviembre de 2021 es electa diputada para...,Nº 20,Región del Bío Bío,Partido Comunista,NaN,NaN,NaN,...,1958-09-12,Femenino,63,NaN,PC,Partido Comunista,0,2022-03-11,2026-03-10 23:59:59,1096_Partido Comunista
1,1097,▪ Casado.,▪ Inicia su trayectoria política al incorporar...,▪ En noviembre de 2021 es electo diputado par...,Nº 20,Región del Bío Bío,Partido Demócrata Cristiano,▪ Administrador Público.,NaN,NaN,...,1968-07-13,Masculino,53,NaN,DC,Partido Demócrata Cristiano,0,2022-03-11,2026-03-10 23:59:59,1097_Partido Demócrata Cristiano
2,1098,▪ Es madre de 4 hijas.,"▪ 2019. Inició su trayectoria política en el ""...",▪ En noviembre de 2021 es electa diputada para...,Nº 3,Región de Antofagasta,Partido Social Cristiano,▪ Estudios Técnicos Agropecuarios.,NaN,NaN,...,1973-02-24,Femenino,49,NaN,IND,Independientes,0,2023-04-19,2025-03-16 23:59:59,1098_Independientes


In [72]:
df_bio_enriquecido["Distrito"] = df_bio_enriquecido["Distrito"].str.extract(r"(\d+)$").astype(int)

In [73]:
nulos = df_bio_enriquecido.isna().sum()
porcentaje = (nulos / len(df_bio_enriquecido)) * 100
nulos_df = pd.DataFrame({"nulos": nulos, "porcentaje": porcentaje.round(2)})
print(nulos_df.sort_values("nulos", ascending=False))

                              nulos  porcentaje
distrito_numero                 226      100.00
Resumen Ejecutivo               225       99.56
Actividades preferidas          223       98.67
Publicaciones                   220       97.35
Idiomas                         218       96.46
Actividades Academicas          209       92.48
Actividades Gremiales           207       91.59
Otras actividades a destacar    176       77.88
Actividades Profesionales       173       76.55
Actividad Laboral               126       55.75
Estudios                         90       39.82
Actividad Política               87       38.50
Antecedentes Familiares          64       28.32
Profesion/Actividad              28       12.39
fecha_nacimiento                  0        0.00
partido_nombre                    0        0.00
militancias_anteriores            0        0.00
Militancia.FechaInicio            0        0.00
Militancia.FechaTermino           0        0.00
partido_id                        0     

In [74]:
df_bio_enriquecido.drop(['distrito_numero', 'Actividades preferidas', 'Resumen Ejecutivo',
                 'Publicaciones', 'Idiomas', 'Actividades Gremiales', 'Actividades Academicas',
                 'Actividades Profesionales', 'Otras actividades a destacar', 'Actividad Laboral',
                 'Partido', 'llave_y', 'llave_x', 'diputado_id', 'periodo_id'], axis=1, inplace=True)

In [75]:
df_bio_enriquecido.head(4)

,id_diputado,Antecedentes Familiares,Actividad Política,Actividad Parlamentaria,Distrito,Región,Profesion/Actividad,Estudios,Periodo,nombre_completo,fecha_nacimiento,sexo,edad,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino
0,1096,▪ Casada. Tiene una hija y un hijo.,▪ Inicia su trayectoria política en las Juvent...,▪ En noviembre de 2021 es electa diputada para...,20,Región del Bío Bío,NaN,NaN,2022-2026,María Candelaria Acevedo Sáez,1958-09-12,Femenino,63,PC,Partido Comunista,0,2022-03-11,2026-03-10 23:59:59
1,1097,▪ Casado.,▪ Inicia su trayectoria política al incorporar...,▪ En noviembre de 2021 es electo diputado par...,20,Región del Bío Bío,▪ Administrador Público.,NaN,2022-2026,Eric Aedo Jeldres,1968-07-13,Masculino,53,DC,Partido Demócrata Cristiano,0,2022-03-11,2026-03-10 23:59:59
2,1098,▪ Es madre de 4 hijas.,"▪ 2019. Inició su trayectoria política en el ""...",▪ En noviembre de 2021 es electa diputada para...,3,Región de Antofagasta,▪ Estudios Técnicos Agropecuarios.,NaN,2022-2026,Yovana Ahumada Palma,1973-02-24,Femenino,49,IND,Independientes,0,2023-04-19,2025-03-16 23:59:59
3,1098,▪ Es madre de 4 hijas.,"▪ 2019. Inició su trayectoria política en el ""...",▪ En noviembre de 2021 es electa diputada para...,3,Región de Antofagasta,▪ Estudios Técnicos Agropecuarios.,NaN,2022-2026,Yovana Ahumada Palma,1973-02-24,Femenino,49,PSC,Partido Social Cristiano,0,2025-03-17,2026-03-10 23:59:59


In [6]:
df_vot = pd.read_csv(DATA_DIR / "2022-2026" / "votaciones_dip.csv")
df_det = pd.read_csv(DATA_DIR / "2022-2026" / "detalle.csv")

In [7]:
df_vot.head(3)

,Unnamed: 0,ID,Fecha,Boletin,Articulo,TotalAfirmativos,TotalNegativos,TotalAbstenciones,TotalDispensados,Tipo.@Codigo,...,Sesion.ID,Sesion.Numero,Sesion.Fecha,Sesion.FechaTermino,Sesion.Tipo.@Codigo,Sesion.Tipo.#text,Tramite.@Codigo,Tramite.#text,Informe.@Codigo,Informe.#text
0,0,39821,2022-12-21T17:22:17,15557-05,Modificaciones incorporadas por el Senado en e...,85,9,4,0,6,...,4287,110,2022-12-21T10:00:00,0001-01-01T00:00:00,60,Ordinaria,3,TERCER TRÁMITE,7,SIN INFORME
1,1,34245,2022-12-14T16:42:55,15557-05,"Artículo 79 del proyecto, cuya votación separa...",108,12,1,1,2,...,4284,107,2022-12-14T10:00:00,0001-01-01T00:00:00,60,Ordinaria,1,PRIMER TRÁMITE,1,PRIMER INFORME
2,2,34244,2022-12-14T16:42:08,15557-05,"Artículo 78 del proyecto, cuya votación separa...",84,31,6,1,2,...,4284,107,2022-12-14T10:00:00,0001-01-01T00:00:00,60,Ordinaria,1,PRIMER TRÁMITE,1,PRIMER INFORME


In [8]:
df_det.head(3)

,Id,Descripcion,Fecha,TotalSi,TotalNo,TotalAbstencion,TotalDispensado,Quorum._value_1,Quorum.Valor,Resultado._value_1,...,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,Diputado.RUT,Diputado.RUTDV,Diputado.Sexo,Diputado.Militancias,OpcionVoto._value_1,OpcionVoto.Valor
0,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Acevedo,Sáez,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
1,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Ahumada,Palma,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
2,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,Alessandri,Vergara,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0


In [9]:
df_vot.shape, df_det.shape

((4140, 25), (770471, 27))

In [10]:
len(df_vot["ID"].unique())

4140

In [11]:
len(df_det["Id"].unique())

6009

In [12]:
df_vot["ID"] = df_vot["ID"].astype(str)
df_det["Id"] = df_det["Id"].astype(str)

df_vot_enriquecido = df_det.merge(
    df_vot,
    left_on="Id",
    right_on="ID",
    how="left"  # <– mantiene solo los diputados de df_bio
)

df_vot_enriquecido.head(3)

,Id,Descripcion,Fecha_x,TotalSi,TotalNo,TotalAbstencion,TotalDispensado,Quorum._value_1,Quorum.Valor,Resultado._value_1,...,Sesion.ID,Sesion.Numero,Sesion.Fecha,Sesion.FechaTermino,Sesion.Tipo.@Codigo,Sesion.Tipo.#text,Tramite.@Codigo,Tramite.#text,Informe.@Codigo,Informe.#text
0,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,4287.0,110.0,2022-12-21T10:00:00,0001-01-01T00:00:00,60.0,Ordinaria,3.0,TERCER TRÁMITE,7.0,SIN INFORME
1,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,4287.0,110.0,2022-12-21T10:00:00,0001-01-01T00:00:00,60.0,Ordinaria,3.0,TERCER TRÁMITE,7.0,SIN INFORME
2,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,4287.0,110.0,2022-12-21T10:00:00,0001-01-01T00:00:00,60.0,Ordinaria,3.0,TERCER TRÁMITE,7.0,SIN INFORME


In [13]:
nulos = df_vot_enriquecido.isna().sum()
porcentaje = (nulos / len(df_vot_enriquecido)) * 100
nulos_df = pd.DataFrame({"nulos": nulos, "porcentaje": porcentaje.round(2)})
print(nulos_df.sort_values("nulos", ascending=False))

                           nulos  porcentaje
Diputado.Nombre2          770471      100.00
Diputado.Militancias      770471      100.00
Diputado.FechaNacimiento  770471      100.00
Diputado.FechaDefucion    770471      100.00
Votos                     770471      100.00
Diputado.RUT              770471      100.00
Diputado.RUTDV            770471      100.00
Diputado.Sexo             770471      100.00
Articulo                  355929       46.20
TotalDispensados          283274       36.77
Tipo.#text                283274       36.77
TotalAbstenciones         283274       36.77
Unnamed: 0                283274       36.77
ID                        283274       36.77
Fecha_y                   283274       36.77
Boletin                   283274       36.77
Tipo.@Codigo              283274       36.77
Informe.#text             283274       36.77
TotalNegativos            283274       36.77
Resultado.@Codigo         283274       36.77
Resultado.#text           283274       36.77
Quorum.@Co

In [14]:
df_vot_enriquecido.drop(['Diputado.Nombre2', 'Diputado.Militancias', 'Diputado.FechaNacimiento',
                        'Diputado.FechaDefucion', 'Votos', 'Diputado.RUT', 'Diputado.RUTDV', 'Diputado.Sexo'], 
                        axis=1, inplace=True)
df_vot_enriquecido.head(3)

,Id,Descripcion,Fecha_x,TotalSi,TotalNo,TotalAbstencion,TotalDispensado,Quorum._value_1,Quorum.Valor,Resultado._value_1,...,Sesion.ID,Sesion.Numero,Sesion.Fecha,Sesion.FechaTermino,Sesion.Tipo.@Codigo,Sesion.Tipo.#text,Tramite.@Codigo,Tramite.#text,Informe.@Codigo,Informe.#text
0,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,4287.0,110.0,2022-12-21T10:00:00,0001-01-01T00:00:00,60.0,Ordinaria,3.0,TERCER TRÁMITE,7.0,SIN INFORME
1,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,4287.0,110.0,2022-12-21T10:00:00,0001-01-01T00:00:00,60.0,Ordinaria,3.0,TERCER TRÁMITE,7.0,SIN INFORME
2,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,4287.0,110.0,2022-12-21T10:00:00,0001-01-01T00:00:00,60.0,Ordinaria,3.0,TERCER TRÁMITE,7.0,SIN INFORME


In [15]:
df_bio_enriquecido.to_csv(DATA_DIR / "2022-2026" / "df_dip_enr.csv")

NameError: name 'df_bio_enriquecido' is not defined

In [16]:
df_bio_enriquecido = pd.read_csv(DATA_DIR / "2022-2026" / "df_dip_enr.csv")

In [17]:
df_vot = df_vot_enriquecido.copy()
df_bio = df_bio_enriquecido.copy()

df_vot["Diputado.Id"] = df_vot["Diputado.Id"].astype(float)
df_vot["Fecha_x"] = pd.to_datetime(df_vot["Fecha_x"])

df_bio["id_diputado"] = df_bio["id_diputado"].astype(float)
df_bio["Militancia.FechaInicio"] = pd.to_datetime(df_bio["Militancia.FechaInicio"])
df_bio["Militancia.FechaTermino"] = pd.to_datetime(df_bio["Militancia.FechaTermino"])


In [18]:
def custom_merge(voto_row):
    """
    Busca la militancia del diputado que cubre la fecha del voto.
    Devuelve una única Series (una fila) con los datos combinados,
    o `None` si no hay coincidencia.
    """
    diputado_id = voto_row["Diputado.Id"]
    fecha_voto  = voto_row["Fecha_x"]

    # Filtrar solo las militancias del diputado
    militancias = df_bio[df_bio["id_diputado"] == diputado_id]

    # Encontrar la militancia que engloba la fecha del voto
    mask = (militancias["Militancia.FechaInicio"] <= fecha_voto) & \
           (fecha_voto <= militancias["Militancia.FechaTermino"])

    # Si hay más de una coincidencia (casos raros) tomamos la primera
    if mask.any():
        militancia_row = militancias[mask].iloc[0]

        # Concatenar las dos Series evitando columnas duplicadas:
        #   - Eliminamos la clave que ya aparece en ambas tablas (ej. id_diputado)
        #   - Renombramos los sufijos para que quede claro el origen
        voto_clean = voto_row.drop(labels=["Diputado.Id"], errors="ignore")
        militancia_clean = militancia_row.drop(labels=["id_diputado"], errors="ignore")

        merged = pd.concat([voto_clean, militancia_clean])
        return merged
    # No hay militancia que cubra la fecha → devolvemos None (se ignora)
    return None

In [23]:
# ---------- Aplicar la función ----------
merged_rows = []
for _, voto in df_vot.iterrows():
    result = custom_merge(voto)
    if result is not None:
        merged_rows.append(result)

In [60]:
# Convertir la lista de Series a DataFrame
df_merged = pd.concat(merged_rows, axis=1).reset_index(drop=True).T

In [62]:
df_merged.columns = merged_rows[0].index

In [64]:
df_merged

,Id,Descripcion,Fecha_x,TotalSi,TotalNo,TotalAbstencion,TotalDispensado,Quorum._value_1,Quorum.Valor,Resultado._value_1,...,Periodo,nombre_completo,fecha_nacimiento,sexo,edad,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino
0,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,2022-2026,María Candelaria Acevedo Sáez,1958-09-12,Femenino,63,PC,Partido Comunista,0,2022-03-11 00:00:00,2026-03-10 23:59:59
0,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,2022-2026,Yovana Ahumada Palma,1973-02-24,Femenino,49,PDG,Partido de la Gente,0,2022-03-11 00:00:00,2023-04-18 23:59:59
1,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,2022-2026,Jorge Alessandri Vergara,1979-06-08,Masculino,42,UDI,Unión Demócrata Independiente,1,2022-03-11 00:00:00,2026-03-10 23:59:59
2,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,2022-2026,René Alinco Bustos,1958-06-02,Masculino,63,IND,Independientes,3,2022-03-11 00:00:00,2026-03-10 23:59:59
3,39821,Boletín N° 15557-05,2022-12-21 17:22:17,85,9,4,0,Quórum Simple,1,Aprobado,...,2022-2026,Jaime Araya Guerrero,1976-02-04,Masculino,46,IND,Independientes,0,2022-03-11 00:00:00,2026-03-10 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742268,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,2022-2026,Nelson Venegas Salazar,1974-04-27,Masculino,47,PS,Partido Socialista,0,2022-03-11 00:00:00,2026-03-10 23:59:59
742269,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,2022-2026,Gastón Von Mühlenbrock Zamora,1954-12-26,Masculino,67,UDI,Unión Demócrata Independiente,4,2022-03-11 00:00:00,2026-03-10 23:59:59
742270,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,2022-2026,Flor Weisse Novoa,1960-03-27,Femenino,61,UDI,Unión Demócrata Independiente,0,2022-03-11 00:00:00,2026-03-10 23:59:59
742271,52568,1-Otros,2025-01-06 17:24:43,63,26,11,0,Quórum Simple,1,Aprobado,...,2022-2026,Gonzalo Winter Etcheberry,1987-01-06,Masculino,35,FA,Frente Amplio,3,2024-07-03 00:00:00,2026-03-10 23:59:59


In [65]:
df_merged.to_csv(DATA_DIR / "2022-2026" / "detalle_enr.csv")